In [1]:
import argparse
import os
import numpy as np
import json
from voc import parse_voc_annotation
from yolo import create_yolov3_model
from generator import BatchGenerator
from utils.utils import normalize, evaluate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
from keras.models import load_model

Using TensorFlow backend.
/home/ehsan/anaconda3/envs/yolo/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ehsan/anaconda3/envs/yolo/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ehsan/anaconda3/envs/yolo/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ehsan/anaconda3/envs/yo

In [10]:
def _main_(args):
    #config_path = args.conf
    config_path = args

    with open(config_path) as config_buffer:    
        config = json.loads(config_buffer.read())

    ###############################
    #   Create the validation generator
    ###############################  
    valid_ints, labels = parse_voc_annotation(
        config['valid']['valid_annot_folder'], 
        config['valid']['valid_image_folder'], 
        config['valid']['cache_name'],
        config['model']['labels']
    )

    labels = labels.keys() if len(config['model']['labels']) == 0 else config['model']['labels']
    labels = sorted(labels)
   
    valid_generator = BatchGenerator(
        instances           = valid_ints, 
        anchors             = config['model']['anchors'],   
        labels              = labels,        
        downsample          = 32, # ratio between network input's size and network output's size, 32 for YOLOv3
        max_box_per_image   = 0,
        batch_size          = config['valid']['batch_size'],
        min_net_size        = config['model']['min_input_size'],
        max_net_size        = config['model']['max_input_size'],   
        shuffle             = True, 
        jitter              = 0.0, 
        norm                = normalize
    )

    ###############################
    #   Load the model and do evaluation
    ###############################
    os.environ['CUDA_VISIBLE_DEVICES'] = config['valid']['gpus']

    infer_model = load_model(config['valid']['saved_weights_name'])

    # compute mAP for all the classes
    average_precisions = evaluate(infer_model, valid_generator)

    # print the score
    for label, average_precision in average_precisions.items():
        print(labels[label] + ': {:.4f}'.format(average_precision))
    print('mAP: {:.4f}'.format(sum(average_precisions.values()) / len(average_precisions)))   

In [11]:
args='config_coco.json'
_main_(args)

aeroplane: 0.0000
apple: 0.0000
backpack: 0.0000
banana: 0.0000
baseball bat: 0.0000
baseball glove: 0.0000
bear: 0.0000
bed: 0.0000
bench: 0.0000
bicycle: 0.0000
bird: 0.0000
boat: 0.0000
book: 0.0000
bottle: 0.0000
bowl: 0.0000
broccoli: 0.0000
bus: 0.0000
cake: 0.0000
car: 0.0000
carrot: 0.0000
cat: 0.0000
cell phone: 0.0000
chair: 0.0000
clock: 0.0000
cow: 0.0000
cup: 0.0000
diningtable: 0.0000
dog: 0.0000
donut: 0.0000
elephant: 0.0000
fire hydrant: 0.0000
fork: 0.0000
frisbee: 0.0000
giraffe: 0.0000
hair drier: 0.0000
handbag: 0.0000
horse: 0.0000
hot dog: 0.0000
keyboard: 0.0000
kite: 0.0000
knife: 0.0000
laptop: 0.0000
microwave: 0.0000
motorbike: 0.0000
mouse: 0.0000
orange: 0.0000
oven: 0.0000
parking meter: 0.0000
person: 0.0000
pizza: 0.0000
pottedplant: 0.0000
refrigerator: 0.0000
remote: 0.0000
sandwich: 0.0000
scissors: 0.0000
sheep: 0.0000
sink: 0.0000
skateboard: 0.0000
skis: 0.0000
snowboard: 0.0000
sofa: 0.0000
spoon: 0.0000
sports ball: 0.0000
stop sign: 0.0000
suit

/home/ehsan/anaconda3/envs/yolo/lib/python3.6/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [ ]:
if __name__ == '__main__':
    argparser = argparse.ArgumentParser(description='Evaluate YOLO_v3 model on any dataset')
    argparser.add_argument('-c', '--conf', help='path to configuration file')    
    
    args = argparser.parse_args()
    _main_(args)